In [ ]:
import os
import re
import glob
import rmgpy.species
import rmgpy.chemkin
from rmgpy.data.thermo import ThermoDatabase

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict



In [ ]:
# load the model
chemkin_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/chem_annotated.inp"
dictionary_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/species_dictionary.txt"
transport_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/tran.dat"
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(
    chemkin_path,
    dictionary_path=dictionary_path,
    transport_path=transport_path
)


In [ ]:
base_dir = "/work/westgroup/harris.se/autoscience/dft/"
species_dirs = glob.glob(os.path.join(base_dir, 'thermo', 'species*'))
rmg_thermos = glob.glob(os.path.join(base_dir, 'thermo', 'species*', 'conformers', 'arkane', 'RMG_libraries', 'thermo.py'))
thermo_libs = glob.glob(os.path.join(base_dir, 'thermo', 'species*', 'conformers', 'arkane', 'RMG_libraries'))

In [ ]:
completed_thermo_indices = []

In [ ]:
print("Thermo completed for the following species:")
for thermo_path in rmg_thermos:
    matches = re.search('species_(.*)/c', thermo_path)
    print(int(matches[1]),"\t", species_list[int(matches[1])])
    completed_thermo_indices.append(int(matches[1]))

In [ ]:
print("Thermo incomplete for the following species:")
for species_dir in species_dirs:
    matches = re.search('species_(.*)', species_dir)
    idx = int(matches[1])
    if idx not in completed_thermo_indices:
        print(idx,"\t", species_list[idx])

In [ ]:
rmg_thermos

In [ ]:
# Load the Arkane thermo
entries = []
for i, lib_path in enumerate(thermo_libs):
    ark_thermo_database = ThermoDatabase()
    ark_thermo_database.load_libraries(
        lib_path,
    )
    try:
        for key in ark_thermo_database.libraries['thermo'].entries.keys():
    #     print(key)
    #     print(ark_thermo_database.libraries['thermo'].entries[key])
            entry = ark_thermo_database.libraries['thermo'].entries[key]
            entry.index = i
            entry.label += f'_{i}'
            entries.append(entry)
    except KeyError:
        print(ark_thermo_database.libraries.keys())

In [ ]:
entries

In [ ]:
ark_thermo_database.libraries['thermo'].entries = OrderedDict()

In [ ]:
for entry in entries:
    ark_thermo_database.libraries['thermo'].entries[entry.label] = entry

In [ ]:
ark_thermo_database.libraries['thermo'].entries

In [ ]:
output_path = "/work/westgroup/harris.se/autoscience/dft/thermo/"
ark_thermo_database.save_libraries(output_path)